# Build a Customer Support Bot

Customer support bots can free up teams' time by handling routine issues, but it can be hard to build a bot that reliably handles diverse tasks in a way that doesn't leave the user pulling their hair out.

In this tutorial, you will build a customer support bot for an airline to help users research and make travel arrangements. You'll learn to use LangGraph's interrupts and checkpointers and more complex state to organize your assistant's tools and manage a user's flight bookings, hotel reservations, car rentals, and excursions. It assumes you are familiar with the concepts presented in the [LangGraph introductory tutorial](https://langchain-ai.github.io/langgraph/tutorials/introduction/).

By the end, you'll have built a working bot and gained an understanding of  LangGraph's key concepts and architectures. You'll be able to apply these design patterns to your other AI projects.

Your final chat bot will look something like the following diagram:

Let's start!

## Prerequisites

First, set up your environment. We'll install this tutorial's prerequisites, download the test DB, and define the tools we will reuse in each section.

We'll be using Claude as our LLM and define a number of custom tools. While most of our tools will connect to a local sqlite database (and require no additional dependencies), we will also provide a general web search to the agent using Tavily.

In [1]:
%%capture --no-stderr
# %pip install -U langgraph langchain-anthropic tavily-python pandas openai
%pip install -e /home/eugene/src/langgraph-tutorials/

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")
_set_env("OPENAI_API_KEY")
# _set_env("TAVILY_API_KEY")

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Tools

Next, define our assistant's tools to search the airline's policy manual and search and manage reservations for flights, hotels, car rentals, and excursions. We will reuse these tools throughout the tutorial. The exact implementations
aren't important, so feel free to run the code below and jump to [Part 1](#part-1-zero-shot).

#### Lookup Company Policies

The assistant retrieve policy information to answer user questions. Note that _enforcement_ of these policies still must be done within the tools/APIs themselves, since the LLM can always ignore this.

In [3]:
from langchain_core.tools import tool
from langgraph_tutorials.customer_support import PolicyRetriever

retriever = PolicyRetriever('openai:text-embedding-3-small')
retriever.initialize()

@tool
def lookup_policy(query: str) -> str:
    """Consult the company policies to check whether certain options are permitted.
    Use this before making any flight changes performing other 'write' events."""
    docs = retriever.query(query, k=2)
    return "\n\n".join([doc["page_content"] for doc in docs])

Initializing embedding model from name: openai:text-embedding-3-small


/home/eugene/src/langgraph-tutorials/langgraph_tutorials/customer_support/policy.py:30: LangChainBetaWarning: The function `init_embeddings` is in beta. It is actively being worked on, so the API may change.
  self._model = init_embeddings(embedding_model)


Fetching FAQ content from https://storage.googleapis.com/benchmarks-artifacts/travel-db/swiss_faq.md...
Embedding documents...
Loaded and embedded 10 documents.


In [34]:
def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

    interrupt = event.get('__interrupt__')
    if interrupt:
        print(interrupt)

In [5]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## Part 1: Simple FAQ Chatbot

We'll start off with a very simple agent that can look up information from a policy.

In [9]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
checkpointer = InMemorySaver()

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[lookup_policy],
    prompt="You are a helpful customer support assistant for Swiss Airlines. Use the provided tools to assist with user's queries.",
    checkpointer=checkpointer
)

In [10]:
import uuid
from langgraph_tutorials.customer_support import DB

DB.initialize()

events = agent.stream(
    {
        "messages": [
            {
                "content": "could you tell me how i can cancel my flight?",
                "role": "user",
            }
        ]
    },
    config={"configurable": { "thread_id": uuid.uuid4() }},
    stream_mode="values",
)

_printed = set()
for event in events:
    _print_event(event, _printed)

================================ Human Message =================================

could you tell me how i can cancel my flight?
================================== Ai Message ==================================

[{'text': "I'd be happy to help you with information about cancelling your flight with Swiss Airlines. Let me check our company policy regarding flight cancellations to provide you with accurate information.", 'type': 'text'}, {'id': 'toolu_018MoMUHPcAZs1csQ4rAzdP8', 'input': {'query': 'flight cancellation process'}, 'name': 'lookup_policy', 'type': 'tool_use'}]
Tool Calls:
  lookup_policy (toolu_018MoMUHPcAZs1csQ4rAzdP8)
 Call ID: toolu_018MoMUHPcAZs1csQ4rAzdP8
  Args:
    query: flight cancellation process
================================= Tool Message =================================
Name: lookup_policy


## How to Cancel a Swiss Air Flight: 877-5O7-7341 Step-by-Step Guide
Swiss Air is a premium airline based in Switzerland that offers a range of domestic and international fl

# Part 2: Add ability to book flights, cars, accommodations etc.

#### Flights

Define the (`fetch_user_flight_information`) tool to let the agent see the current user's flight information.  Then define tools to search for flights and manage the passenger's bookings stored in the SQL database.

We then can [access the RunnableConfig](https://python.langchain.com/docs/how_to/tool_configure/#inferring-by-parameter-type) for a given run to check the `passenger_id` of the user accessing this application. The LLM never has to provide these explicitly, they are provided for a given invocation of the graph so that each user cannot access other passengers' booking information.

<div class="admonition warning">
    <p class="admonition-title">Compatibility</p>
    <p>
        This tutorial expects `langchain-core>=0.2.16` to use the injected RunnableConfig. Prior to that, you'd use `ensure_config` to collect the config from context.
    </p>
</div> 


#### Car Rental Tools

Once a user books a flight, they likely will want to organize transportation. Define some "car rental" tools to let the user search for and reserve a car at their destination.

#### Hotels

The user has to sleep! Define some tools to search for and manage hotel reservations.

#### Excursions

Finally, define some tools to let the user search for things to do (and make reservations) once they arrive.

#### Utilities

Define helper functions to pretty print the messages in the graph while we debug it and to give our tool node error handling (by adding the error to the chat history).

In [21]:
from langgraph_tutorials.customer_support.flights import search_flights, cancel_ticket, fetch_user_flight_information, update_ticket_to_new_flight
from langgraph_tutorials.customer_support.cars import search_car_rentals, book_car_rental, update_car_rental, cancel_car_rental
from langgraph_tutorials.customer_support.excursions import search_trip_recommendations, book_excursion, update_excursion, cancel_excursion
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import AnyMessage
from datetime import datetime

tools = [
    # Flights
    search_flights, cancel_ticket, fetch_user_flight_information, update_ticket_to_new_flight,
    # Car rentals
    search_car_rentals, book_car_rental, update_car_rental, cancel_car_rental,
    # Excursions
    search_trip_recommendations, book_excursion, update_excursion, cancel_excursion,
    # Company policy
    lookup_policy,
]


def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    time = datetime.now
    system_message = (
        "You are a helpful customer support assistant for Swiss Airlines. "
        " Use the provided tools to search for flights, company policies, and other information to assist the user's queries. "
        " When searching, be persistent. Expand your query bounds if the first search returns no results. "
        " If a search comes up empty, expand your search before giving up."
        f"\nCurrent time: {time}.",
    )
    return [{"role": "system", "content": system_message}] + state["messages"]


# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
checkpointer = InMemorySaver()

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=tools,
    prompt=prompt,
    checkpointer=checkpointer
)

In [23]:
import uuid

# Let's create an example conversation a user might have with the assistant
tutorial_questions = [
    "cancel my current flight",
]

# Update with the backup file so we can restart from the original place in each section
DB.initialize()

thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "passenger_id": "3442 587242",
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}


_printed = set()
for question in tutorial_questions:
    events = agent.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)

================================ Human Message =================================

cancel my current flight
================================== Ai Message ==================================

[{'text': "I'd be happy to help you cancel your current flight. To proceed, I'll need to first check your current flight information and then verify our cancellation policy.\n\nLet me retrieve your current flight information:", 'type': 'text'}, {'id': 'toolu_01QTEX5drUemwQurLvQ3XQEc', 'input': {}, 'name': 'fetch_user_flight_information', 'type': 'tool_use'}]
Tool Calls:
  fetch_user_flight_information (toolu_01QTEX5drUemwQurLvQ3XQEc)
 Call ID: toolu_01QTEX5drUemwQurLvQ3XQEc
  Args:
================================= Tool Message =================================
Name: fetch_user_flight_information

[{"ticket_no": "7240005432906569", "book_ref": "C46E9F", "flight_id": 19250, "flight_no": "LX0112", "departure_airport": "CDG", "arrival_airport": "BSL", "scheduled_departure": "2025-05-19 15:47:31.612017-0

#### Example Conversation

Now it's time to try out our mighty chatbot! Let's run it over the following list of dialog turns. If it hits a "RecursionLimit", that means the agent wasn't able to get an answer in the allocated number of steps. That's OK! We have more tricks up our sleeve in later sections of this tutorial.

In [81]:
import uuid

# Let's create an example conversation a user might have with the assistant
tutorial_questions = [
    "Hi there, what time is my flight?",
    "Am i allowed to update my flight to something sooner? I want to leave later today.",
    "Update my flight to sometime next week then",
    "The next available option is great",
    "what about lodging and transportation?",
    "Yeah i think i'd like an affordable hotel for my week-long stay (7 days). And I'll want to rent a car.",
    "OK could you place a reservation for your recommended hotel? It sounds nice.",
    "yes go ahead and book anything that's moderate expense and has availability.",
    "Now for a car, what are my options?",
    "Awesome let's just get the cheapest option. Go ahead and book for 7 days",
    "Cool so now what recommendations do you have on excursions?",
    "Are they available while I'm there?",
    "interesting - i like the museums, what options are there? ",
    "OK great pick one and book it for my second day there.",
]

# Update with the backup file so we can restart from the original place in each section
DB.initialize()


thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "passenger_id": "3442 587242",
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}


_printed = set()
for question in tutorial_questions:
    events = agent.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)

================================ Human Message =================================

Hi there, what time is my flight?
================================== Ai Message ==================================

[{'text': "Hello! I'd be happy to help you check your flight information. To provide you with accurate details about your upcoming flight, I'll need to look up your flight information.\n\nLet me fetch your flight details:", 'type': 'text'}, {'id': 'toolu_0157GDR9kgFbkJfVDs8guxbW', 'input': {}, 'name': 'fetch_user_flight_information', 'type': 'tool_use'}]
Tool Calls:
  fetch_user_flight_information (toolu_0157GDR9kgFbkJfVDs8guxbW)
 Call ID: toolu_0157GDR9kgFbkJfVDs8guxbW
  Args:
================================= Tool Message =================================
Name: fetch_user_flight_information

[{"ticket_no": "7240005432906569", "book_ref": "C46E9F", "flight_id": 19250, "flight_no": "LX0112", "departure_airport": "CDG", "arrival_airport": "BSL", "scheduled_departure": "2025-05-19 12:38:30.020

#### Part 1 Review

Our simple assistant is not bad! It was able to respond reasonably well for all the questions, quickly respond in-context, and successfully execute all our tasks.

If this were a simple Q&A bot, we'd probably be happy with the results above. Since our customer support bot is taking actions on behalf of the user, some of its behavior above is a bit concerning:

1. The assistant booked a car when we were focusing on lodging, then had to cancel and rebook later on: oops! The user should have final say before booking to avoid unwanted feeds.
2. The assistant struggled to search for recommendations. We could improve this by adding more verbose instructions and examples using the tool, but doing this for every tool can lead to a large prompt and overwhelmed agent.
3. The assistant had to do an explicit search just to get the user's relevant information. We can save a lot of time by fetching the user's relevant travel details immediately so the assistant can directly respond.

In the next section, we will address the first two of these issues.

## Part 2: Add Confirmation

When an assistant takes actions on behalf of the user, the user should (almost) always have the final say on whether to follow through with the actions. Otherwise, any small mistake the assistant makes (or any prompt injection it succombs to) can cause real damage to the user.

In this section, we will use `interrupt_before` to pause the graph and return control to the user **before** executing any of the tools.

Your graph will look something like the following:

As before, start by defining the state:

#### State & Assistant

Our graph state and LLM calling is nearly identical to Part 1 except Exception:

- We've added a `user_info` field that will be eagerly populated by our graph
- We can use the state directly in the `Assistant` object rather than using the configurable params

In [35]:
from langgraph_tutorials.customer_support.flights import search_flights, cancel_ticket, fetch_user_flight_information, update_ticket_to_new_flight
from langgraph_tutorials.customer_support.cars import search_car_rentals, book_car_rental, update_car_rental, cancel_car_rental
from langgraph_tutorials.customer_support.excursions import search_trip_recommendations, book_excursion, update_excursion, cancel_excursion
from langgraph.prebuilt.chat_agent_executor import AgentState

from langchain_core.runnables import RunnableConfig
from langchain_core.messages import AnyMessage
from datetime import datetime
from langgraph.prebuilt.interrupt import HumanInterruptConfig, InterruptToolNode

# These are tools for which we'd like user in the loop approval
sensitive_tools = [
    cancel_ticket, update_ticket_to_new_flight,
    book_car_rental, update_car_rental, cancel_car_rental,
    book_excursion, update_excursion, cancel_excursion,
        fetch_user_flight_information,
]

human_config = {
    tool.name: HumanInterruptConfig(
          allow_accept=True,
          allow_edit=False,
          allow_ignore=False,
          allow_respond=True,
      )
    for tool in sensitive_tools
}

safe_tools = [
    # Flights
    search_flights,
    # Car rentals
    search_car_rentals, 
    # Excursions
    search_trip_recommendations, 
    # Company policy
    lookup_policy,
    # Fetch user flight
    # fetch_user_flight_information,
]

all_tools = sensitive_tools + safe_tools

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    time = datetime.now
    system_message = (
        "You are a helpful customer support assistant for Swiss Airlines. "
        " Use the provided tools to search for flights, company policies, and other information to assist the user's queries. "
        " When searching, be persistent. Expand your query bounds if the first search returns no results. "
        " If a search comes up empty, expand your search before giving up."
        f"\nCurrent time: {time}.",
    )
    return [{"role": "system", "content": system_message}] + state["messages"]


# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
checkpointer = InMemorySaver()

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=tools,
    prompt=prompt,
    checkpointer=checkpointer,
    post_model_hook=InterruptToolNode(
        **human_config
    ),
)

In [39]:
import uuid
from langgraph_tutorials.customer_support import DB

DB.initialize()

thread_id = uuid.uuid4()

In [40]:
events = agent.stream(
    {
        "messages": [
            {
                "content": "cancel my flight",
                "role": "user",
            }
        ]
    },
    config = {
        "configurable": {
            # The passenger_id is used in our flight tools to
            # fetch the user's flight information
            "passenger_id": "3442 587242",
            # Checkpoints are accessed by thread_id
            "thread_id": thread_id,
        }
    },
    stream_mode="values",
)

_printed = set()
for event in events:
    # print(event)
    _print_event(event, _printed)


================================ Human Message =================================

cancel my flight
================================== Ai Message ==================================

[{'text': "I'd be happy to help you cancel your flight, but I'll need to retrieve your flight information first. Let me check what flights you currently have booked.", 'type': 'text'}, {'id': 'toolu_01SUbVbYLpKKZUG6eNtQEZs3', 'input': {}, 'name': 'fetch_user_flight_information', 'type': 'tool_use'}]
Tool Calls:
  fetch_user_flight_information (toolu_01SUbVbYLpKKZUG6eNtQEZs3)
 Call ID: toolu_01SUbVbYLpKKZUG6eNtQEZs3
  Args:
(Interrupt(value=[{'action_request': {'action': 'fetch_user_flight_information', 'args': {}}, 'config': {'allow_accept': True, 'allow_edit': True, 'allow_ignore': True, 'allow_respond': True}, 'description': 'Please review tool call for `fetch_user_flight_information` before execution.'}], resumable=True, ns=['post_model_hook:a3582491-f1f3-f62d-d321-64aeb55ff6a0']),)


In [45]:
if '__interrupt__' in event:
    agent.invoke

#### Example Conversation

Now it's time to try out our newly revised chatbot! Let's run it over the following list of dialog turns.

In [ ]:
# import shutil
# import uuid

# # Update with the backup file so we can restart from the original place in each section
# db = update_dates(db)
# thread_id = str(uuid.uuid4())

# config = {
#     "configurable": {
#         # The passenger_id is used in our flight tools to
#         # fetch the user's flight information
#         "passenger_id": "3442 587242",
#         # Checkpoints are accessed by thread_id
#         "thread_id": thread_id,
#     }
# }


# _printed = set()
# # We can reuse the tutorial questions from part 1 to see how it does.
# for question in tutorial_questions:
#     events = part_2_graph.stream(
#         {"messages": ("user", question)}, config, stream_mode="values"
#     )
#     for event in events:
#         _print_event(event, _printed)
#     snapshot = part_2_graph.get_state(config)
#     while snapshot.next:
#         # We have an interrupt! The agent is trying to use a tool, and the user can approve or deny it
#         # Note: This code is all outside of your graph. Typically, you would stream the output to a UI.
#         # Then, you would have the frontend trigger a new run via an API call when the user has provided input.
#         try:
#             user_input = input(
#                 "Do you approve of the above actions? Type 'y' to continue;"
#                 " otherwise, explain your requested changed.\n\n"
#             )
#         except:
#             user_input = "y"
#         if user_input.strip() == "y":
#             # Just continue
#             result = part_2_graph.invoke(
#                 None,
#                 config,
#             )
#         else:
#             # Satisfy the tool invocation by
#             # providing instructions on the requested changes / change of mind
#             result = part_2_graph.invoke(
#                 {
#                     "messages": [
#                         ToolMessage(
#                             tool_call_id=event["messages"][-1].tool_calls[0]["id"],
#                             content=f"API call denied by user. Reasoning: '{user_input}'. Continue assisting, accounting for the user's input.",
#                         )
#                     ]
#                 },
#                 config,
#             )
#         snapshot = part_2_graph.get_state(config)

Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


================================ Human Message =================================

The next available option is great
================================== Ai Message ==================================

[{'text': "Got it, let's update your ticket to the next available Swiss Air flight from Paris (CDG) to Basel (BSL) next week.\n\nBased on the search results, the next available flight after your originally scheduled one is:\n\nFlight No: LX0112\nDeparture: 2024-05-01 20:37 (CDG) \nArrival: 2024-05-01 22:07 (BSL)\nFlight ID: 19233\n\nLet me confirm the policy allows updating to this new flight date and time with your Economy Flex ticket.", 'type': 'text'}, {'id': 'toolu_01YBwigKSeqeELNRa66B8iST', 'input': {'query': 'changing economy flex ticket to different date'}, 'name': 'lookup_policy', 'type': 'tool_use'}]
Tool Calls:
  lookup_policy (toolu_01YBwigKSeqeELNRa66B8iST)
 Call ID: toolu_01YBwigKSeqeELNRa66B8iST
  Args:
    query: changing economy flex ticket to different date


Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y
Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


================================ Human Message =================================

what about lodging and transportation?
================================== Ai Message ==================================

[{'text': 'Sure, let me help you with arranging lodging and transportation for your updated travel dates in Basel next week.\n\nFor hotels, we can search and book accommodations during your stay:', 'type': 'text'}, {'id': 'toolu_01PBJ6rZ2P9tvVLWPt5Nrck7', 'input': {'checkin_date': '2024-05-01', 'checkout_date': '2024-05-02', 'location': 'Basel'}, 'name': 'search_hotels', 'type': 'tool_use'}]
Tool Calls:
  search_hotels (toolu_01PBJ6rZ2P9tvVLWPt5Nrck7)
 Call ID: toolu_01PBJ6rZ2P9tvVLWPt5Nrck7
  Args:
    checkin_date: 2024-05-01
    checkout_date: 2024-05-02
    location: Basel


Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y
Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


...

Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y
Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


================================ Human Message =================================

Are they available while I'm there?
================================== Ai Message ==================================

[{'text': 'Good point, let me verify availability for those recommended excursions during your stay in Basel from May 1st to May 8th.', 'type': 'text'}, {'id': 'toolu_019wuQZVgGoNPcJDofm2zETY', 'input': {'location': 'Basel'}, 'name': 'search_trip_recommendations', 'type': 'tool_use'}]
Tool Calls:
  search_trip_recommendations (toolu_019wuQZVgGoNPcJDofm2zETY)
 Call ID: toolu_019wuQZVgGoNPcJDofm2zETY
  Args:
    location: Basel


Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


================================ Human Message =================================

interesting - i like the museums, what options are there? OK great pick one and book it for my second day there.
================================== Ai Message ==================================

[{'text': "Sounds good, let's book the Kunstmuseum Basel art museum for your second day in the city on May 2nd.", 'type': 'text'}, {'id': 'toolu_01F4EQx4PFJDcdHRFgSSVdEf', 'input': {'recommendation_id': 2}, 'name': 'book_excursion', 'type': 'tool_use'}]
Tool Calls:
  book_excursion (toolu_01F4EQx4PFJDcdHRFgSSVdEf)
 Call ID: toolu_01F4EQx4PFJDcdHRFgSSVdEf
  Args:
    recommendation_id: 2


Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


#### Part 2 Review

Now our assistant was able to save a step to respond with our flight details. We also completely controlled which actions were performed. This all worked using LangGraph's `interrupts` and `checkpointers`. The interrupt pauses graph execution, its state safely persisted using your configured checkpointer. The user can then start it up at any time by running it with the right config.

See an [example LangSmith trace](https://smith.langchain.com/public/b3c71814-c366-476d-be6a-f6f3056caaec/r) to get a better sense of how the graph is running. Note [from this trace](https://smith.langchain.com/public/a077f4be-6baa-4e97-89f7-0dabc65c0fd0/r) that you typically **resume** a flow by invoking the graph with `(None, config)`. The state is loaded from the checkpoint as if it never was interrupted.

This graph worked pretty well! We *didn't really* need to be involved in *EVERY* assistant action, though...

In the next section, we will reorganize our graph so that we can interrupt only on the "sensitive" actions that actually write to the database.

In [ ]:
import shutil
import uuid

# Update with the backup file so we can restart from the original place in each section
db = update_dates(db)
thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "passenger_id": "3442 587242",
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

tutorial_questions = [
    "Hi there, what time is my flight?",
    "Am i allowed to update my flight to something sooner? I want to leave later today.",
    "Update my flight to sometime next week then",
    "The next available option is great",
    "what about lodging and transportation?",
    "Yeah i think i'd like an affordable hotel for my week-long stay (7 days). And I'll want to rent a car.",
    "OK could you place a reservation for your recommended hotel? It sounds nice.",
    "yes go ahead and book anything that's moderate expense and has availability.",
    "Now for a car, what are my options?",
    "Awesome let's just get the cheapest option. Go ahead and book for 7 days",
    "Cool so now what recommendations do you have on excursions?",
    "Are they available while I'm there?",
    "interesting - i like the museums, what options are there? ",
    "OK great pick one and book it for my second day there.",
]


_printed = set()
# We can reuse the tutorial questions from part 1 to see how it does.
for question in tutorial_questions:
    events = part_3_graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)
    snapshot = part_3_graph.get_state(config)
    while snapshot.next:
        # We have an interrupt! The agent is trying to use a tool, and the user can approve or deny it
        # Note: This code is all outside of your graph. Typically, you would stream the output to a UI.
        # Then, you would have the frontend trigger a new run via an API call when the user has provided input.
        try:
            user_input = input(
                "Do you approve of the above actions? Type 'y' to continue;"
                " otherwise, explain your requested changed.\n\n"
            )
        except:
            user_input = "y"
        if user_input.strip() == "y":
            # Just continue
            result = part_3_graph.invoke(
                None,
                config,
            )
        else:
            # Satisfy the tool invocation by
            # providing instructions on the requested changes / change of mind
            result = part_3_graph.invoke(
                {
                    "messages": [
                        ToolMessage(
                            tool_call_id=event["messages"][-1].tool_calls[0]["id"],
                            content=f"API call denied by user. Reasoning: '{user_input}'. Continue assisting, accounting for the user's input.",
                        )
                    ]
                },
                config,
            )
        snapshot = part_3_graph.get_state(config)

================================ Human Message =================================

OK could you place a reservation for your recommended hotel? It sounds nice.
================================== Ai Message ==================================

[{'text': "Sure, I'd be happy to book the Hilton Basel hotel for your stay since it seems like you're interested in that luxury option.\n\nJust to confirm the details:\n\nHotel: Hilton Basel\nLocation: Basel, Switzerland \nCheck-in: May 2nd, 2024\nCheck-out: May 9th, 2024 \nTotal Nights: 7\n\nThe Hilton Basel is a 5-star luxury hotel located right on the River Rhine. It has an indoor pool, spa, fitness center and multiple dining options on site.", 'type': 'text'}, {'id': 'toolu_01P4J1WqwRTTdY9LTumMCewh', 'input': {'hotel_id': 1}, 'name': 'book_hotel', 'type': 'tool_use'}]
Tool Calls:
  book_hotel (toolu_01P4J1WqwRTTdY9LTumMCewh)
 Call ID: toolu_01P4J1WqwRTTdY9LTumMCewh
  Args:
    hotel_id: 1


Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


================================ Human Message =================================

yes go ahead and book anything that's moderate expense and has availability.
================================== Ai Message ==================================

[{'text': "Got it, no problem. For your upcoming trip to Basel, I'll aim for moderately priced but good quality options that are available for your dates. \n\nLet me revise the hotel and rental car bookings:\n\nHotel:", 'type': 'text'}, {'id': 'toolu_01Rj5vmxjSztKxKimH7VYEoc', 'input': {'checkin_date': '2024-05-02', 'checkout_date': '2024-05-09', 'location': 'Basel', 'price_tier': 'Upscale'}, 'name': 'search_hotels', 'type': 'tool_use'}]
Tool Calls:
  search_hotels (toolu_01Rj5vmxjSztKxKimH7VYEoc)
 Call ID: toolu_01Rj5vmxjSztKxKimH7VYEoc
  Args:
    checkin_date: 2024-05-02
    checkout_date: 2024-05-09
    location: Basel
    price_tier: Upscale
================================= Tool Message =================================
Name: search_hotels

[{

Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


================================ Human Message =================================

Now for a car, what are my options?
================================== Ai Message ==================================

[{'text': 'Sure, let me search for car rental options in Basel for your dates of May 2nd to May 9th:', 'type': 'text'}, {'id': 'toolu_01KRkZuw1z7BxChERpVuGVZB', 'input': {'end_date': '2024-05-09', 'location': 'Basel', 'start_date': '2024-05-02'}, 'name': 'search_car_rentals', 'type': 'tool_use'}]
Tool Calls:
  search_car_rentals (toolu_01KRkZuw1z7BxChERpVuGVZB)
 Call ID: toolu_01KRkZuw1z7BxChERpVuGVZB
  Args:
    end_date: 2024-05-09
    location: Basel
    start_date: 2024-05-02
================================= Tool Message =================================
Name: search_car_rentals

[{"id": 1, "name": "Europcar", "location": "Basel", "price_tier": "Economy", "start_date": "2024-04-14", "end_date": "2024-04-11", "booked": 1}, {"id": 2, "name": "Avis", "location": "Basel", "price_tier": "L

Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


================================ Human Message =================================

Cool so now what recommendations do you have on excursions?
================================== Ai Message ==================================

[{'text': 'Great, let me look into some recommended excursions and activities to do during your week-long stay in Basel:', 'type': 'text'}, {'id': 'toolu_01CdRKsURqjvbTtLyBMQcQtM', 'input': {'location': 'Basel'}, 'name': 'search_trip_recommendations', 'type': 'tool_use'}]
Tool Calls:
  search_trip_recommendations (toolu_01CdRKsURqjvbTtLyBMQcQtM)
 Call ID: toolu_01CdRKsURqjvbTtLyBMQcQtM
  Args:
    location: Basel
================================= Tool Message =================================
Name: search_trip_recommendations

[{"id": 1, "name": "Basel Minster", "location": "Basel", "keywords": "landmark, history", "details": "Visit the historic Basel Minster, a beautiful Gothic cathedral.", "booked": 0}, {"id": 2, "name": "Kunstmuseum Basel", "location": "Basel", "

Do you approve of the above actions? Type 'y' to continue; otherwise, explain your requested changed.

 y


#### Part 3 Review

Much better! Our agent is now working well - [check out a LangSmith trace](https://smith.langchain.com/public/a0d64d8b-1714-4cfe-a239-e170ca45e81a/r) of our latest run to inspect its work! You may be satisfied with this design. The code is contained, and it's behaving as desired. 

One problem with this design is that we're putting a lot of pressure on a single prompt. If we want to add more tools, or if each tool gets more complicated (more filters, more business logic constraining behavior, etc), it's likely the tool usage and overall behavior of the bot will start to suffer. 

In the next section, we show how you can take more control over different user experiences by routing to specialist agents or sub-graphs based on the user's intent.

## Part 4: Specialized Workflows

In the previous sections, we saw how "wide" chat-bots, relying on a single prompt and LLM to handle various user intents, can get us far. However, it's difficult to create **predictably great** user experiences for known intents with this approach.

Alternatively, your graph can detect userintent and select the appropriate workflow or "skill" to satisfy the user's needs. Each workflow can focus on its domain, allowing for isolated improvements without degrading the overall assistant.

In this section, we'll split user experiences into separate sub-graphs, resulting in a structure like this:

<!-- <img src="./img/part-4-diagram.png" src="../img/part-4-diagram.png"> -->

In the diagram above, each square wraps an agentic, focused workflow. The primary assistant fields the user's initial queries, and the graph routes to the appropriate "expert" based on the query content.

#### State

We want to keep track of which sub-graph is in control at any given moment. While we _could_ do this through some arithmetic on the message list, it's easier to track as a dedicated **stack**. 

Add a `dialog_state` list to the `State` below. Any time a `node` is run and returns a value for `dialog_state`, the `update_dialog_stack` function will be called to determine how to apply the update.

<!-- #### Assistants

This time we will create an assistant **for every workflow**. That means:

1. Flight booking assistant
2. Hotel booking assistant
3. Car rental assistant
4. Excursion assistant
5. and finally, a "primary assistant" to route between these

If you're paying attention, you may recognize this as an example of the **supervisor** design pattern from our Multi-agent examples.

Below, define the `Runnable` objects to power each assistant.
Each `Runnable` has a prompt, LLM, and schemas for the tools scoped to that assistant.
Each *specialized* / delegated assistant additionally can call the `CompleteOrEscalate` tool to indicate that the control flow should be passed back to the primary assistant. This happens if it has successfully completed its work or if the user has changed their mind or needs assistance on something that beyond the scope of that particular workflow. -->

<!-- from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig

from pydantic import BaseModel, Field


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            result = self.runnable.invoke(state)

            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


class CompleteOrEscalate(BaseModel):
    """A tool to mark the current task as completed and/or to escalate control of the dialog to the main assistant,
    who can re-route the dialog based on the user's needs."""

    cancel: bool = True
    reason: str

    class Config:
        json_schema_extra = {
            "example": {
                "cancel": True,
                "reason": "User changed their mind about the current task.",
            },
            "example 2": {
                "cancel": True,
                "reason": "I have fully completed the task.",
            },
            "example 3": {
                "cancel": False,
                "reason": "I need to search the user's emails or calendar for more information.",
            },
        }


# Flight booking assistant

flight_booking_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a specialized assistant for handling flight updates. "
            " The primary assistant delegates work to you whenever the user needs help updating their bookings. "
            "Confirm the updated flight details with the customer and inform them of any additional fees. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            "If you need more information or the customer changes their mind, escalate the task back to the main assistant."
            " Remember that a booking isn't completed until after the relevant tool has successfully been used."
            "\n\nCurrent user flight information:\n<Flights>\n{user_info}\n</Flights>"
            "\nCurrent time: {time}."
            "\n\nIf the user needs help, and none of your tools are appropriate for it, then"
            ' "CompleteOrEscalate" the dialog to the host assistant. Do not waste the user\'s time. Do not make up invalid tools or functions.',
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

update_flight_safe_tools = [search_flights]
update_flight_sensitive_tools = [update_ticket_to_new_flight, cancel_ticket]
update_flight_tools = update_flight_safe_tools + update_flight_sensitive_tools
update_flight_runnable = flight_booking_prompt | llm.bind_tools(
    update_flight_tools + [CompleteOrEscalate]
)

# Hotel Booking Assistant
book_hotel_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a specialized assistant for handling hotel bookings. "
            "The primary assistant delegates work to you whenever the user needs help booking a hotel. "
            "Search for available hotels based on the user's preferences and confirm the booking details with the customer. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            "If you need more information or the customer changes their mind, escalate the task back to the main assistant."
            " Remember that a booking isn't completed until after the relevant tool has successfully been used."
            "\nCurrent time: {time}."
            '\n\nIf the user needs help, and none of your tools are appropriate for it, then "CompleteOrEscalate" the dialog to the host assistant.'
            " Do not waste the user's time. Do not make up invalid tools or functions."
            "\n\nSome examples for which you should CompleteOrEscalate:\n"
            " - 'what's the weather like this time of year?'\n"
            " - 'nevermind i think I'll book separately'\n"
            " - 'i need to figure out transportation while i'm there'\n"
            " - 'Oh wait i haven't booked my flight yet i'll do that first'\n"
            " - 'Hotel booking confirmed'",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

book_hotel_safe_tools = [search_hotels]
book_hotel_sensitive_tools = [book_hotel, update_hotel, cancel_hotel]
book_hotel_tools = book_hotel_safe_tools + book_hotel_sensitive_tools
book_hotel_runnable = book_hotel_prompt | llm.bind_tools(
    book_hotel_tools + [CompleteOrEscalate]
)

# Car Rental Assistant
book_car_rental_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a specialized assistant for handling car rental bookings. "
            "The primary assistant delegates work to you whenever the user needs help booking a car rental. "
            "Search for available car rentals based on the user's preferences and confirm the booking details with the customer. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            "If you need more information or the customer changes their mind, escalate the task back to the main assistant."
            " Remember that a booking isn't completed until after the relevant tool has successfully been used."
            "\nCurrent time: {time}."
            "\n\nIf the user needs help, and none of your tools are appropriate for it, then "
            '"CompleteOrEscalate" the dialog to the host assistant. Do not waste the user\'s time. Do not make up invalid tools or functions.'
            "\n\nSome examples for which you should CompleteOrEscalate:\n"
            " - 'what's the weather like this time of year?'\n"
            " - 'What flights are available?'\n"
            " - 'nevermind i think I'll book separately'\n"
            " - 'Oh wait i haven't booked my flight yet i'll do that first'\n"
            " - 'Car rental booking confirmed'",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

book_car_rental_safe_tools = [search_car_rentals]
book_car_rental_sensitive_tools = [
    book_car_rental,
    update_car_rental,
    cancel_car_rental,
]
book_car_rental_tools = book_car_rental_safe_tools + book_car_rental_sensitive_tools
book_car_rental_runnable = book_car_rental_prompt | llm.bind_tools(
    book_car_rental_tools + [CompleteOrEscalate]
)

# Excursion Assistant

book_excursion_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a specialized assistant for handling trip recommendations. "
            "The primary assistant delegates work to you whenever the user needs help booking a recommended trip. "
            "Search for available trip recommendations based on the user's preferences and confirm the booking details with the customer. "
            "If you need more information or the customer changes their mind, escalate the task back to the main assistant."
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " Remember that a booking isn't completed until after the relevant tool has successfully been used."
            "\nCurrent time: {time}."
            '\n\nIf the user needs help, and none of your tools are appropriate for it, then "CompleteOrEscalate" the dialog to the host assistant. Do not waste the user\'s time. Do not make up invalid tools or functions.'
            "\n\nSome examples for which you should CompleteOrEscalate:\n"
            " - 'nevermind i think I'll book separately'\n"
            " - 'i need to figure out transportation while i'm there'\n"
            " - 'Oh wait i haven't booked my flight yet i'll do that first'\n"
            " - 'Excursion booking confirmed!'",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

book_excursion_safe_tools = [search_trip_recommendations]
book_excursion_sensitive_tools = [book_excursion, update_excursion, cancel_excursion]
book_excursion_tools = book_excursion_safe_tools + book_excursion_sensitive_tools
book_excursion_runnable = book_excursion_prompt | llm.bind_tools(
    book_excursion_tools + [CompleteOrEscalate]
)


# Primary Assistant
class ToFlightBookingAssistant(BaseModel):
    """Transfers work to a specialized assistant to handle flight updates and cancellations."""

    request: str = Field(
        description="Any necessary followup questions the update flight assistant should clarify before proceeding."
    )


class ToBookCarRental(BaseModel):
    """Transfers work to a specialized assistant to handle car rental bookings."""

    location: str = Field(
        description="The location where the user wants to rent a car."
    )
    start_date: str = Field(description="The start date of the car rental.")
    end_date: str = Field(description="The end date of the car rental.")
    request: str = Field(
        description="Any additional information or requests from the user regarding the car rental."
    )

    class Config:
        json_schema_extra = {
            "example": {
                "location": "Basel",
                "start_date": "2023-07-01",
                "end_date": "2023-07-05",
                "request": "I need a compact car with automatic transmission.",
            }
        }


class ToHotelBookingAssistant(BaseModel):
    """Transfer work to a specialized assistant to handle hotel bookings."""

    location: str = Field(
        description="The location where the user wants to book a hotel."
    )
    checkin_date: str = Field(description="The check-in date for the hotel.")
    checkout_date: str = Field(description="The check-out date for the hotel.")
    request: str = Field(
        description="Any additional information or requests from the user regarding the hotel booking."
    )

    class Config:
        json_schema_extra = {
            "example": {
                "location": "Zurich",
                "checkin_date": "2023-08-15",
                "checkout_date": "2023-08-20",
                "request": "I prefer a hotel near the city center with a room that has a view.",
            }
        }


class ToBookExcursion(BaseModel):
    """Transfers work to a specialized assistant to handle trip recommendation and other excursion bookings."""

    location: str = Field(
        description="The location where the user wants to book a recommended trip."
    )
    request: str = Field(
        description="Any additional information or requests from the user regarding the trip recommendation."
    )

    class Config:
        json_schema_extra = {
            "example": {
                "location": "Lucerne",
                "request": "The user is interested in outdoor activities and scenic views.",
            }
        }


# The top-level assistant performs general Q&A and delegates specialized tasks to other assistants.
# The task delegation is a simple form of semantic routing / does simple intent detection
# llm = ChatAnthropic(model="claude-3-haiku-20240307")
llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=1)

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful customer support assistant for Swiss Airlines. "
            "Your primary role is to search for flight information and company policies to answer customer queries. "
            "If a customer requests to update or cancel a flight, book a car rental, book a hotel, or get trip recommendations, "
            "delegate the task to the appropriate specialized assistant by invoking the corresponding tool. You are not able to make these types of changes yourself."
            " Only the specialized assistants are given permission to do this for the user."
            "The user is not aware of the different specialized assistants, so do not mention them; just quietly delegate through function calls. "
            "Provide detailed information to the customer, and always double-check the database before concluding that information is unavailable. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " If a search comes up empty, expand your search before giving up."
            "\n\nCurrent user flight information:\n<Flights>\n{user_info}\n</Flights>"
            "\nCurrent time: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)
primary_assistant_tools = [
    TavilySearchResults(max_results=1),
    search_flights,
    lookup_policy,
]
assistant_runnable = primary_assistant_prompt | llm.bind_tools(
    primary_assistant_tools
    + [
        ToFlightBookingAssistant,
        ToBookCarRental,
        ToHotelBookingAssistant,
        ToBookExcursion,
    ]
)

#### Create Assistant

We're about ready to create the graph. In the previous section, we made the design decision to have a shared `messages` state between all the nodes. This is powerful in that each delegated assistant can see the entire user journey and have a shared context. This, however, means that weaker LLMs can easily get mixed up about there specific scope. To mark the "handoff" between the primary assistant and one of the delegated workflows (and complete the tool call from the router), we will add a `ToolMessage` to the state.


#### Utility

Create a function to make an "entry" node for each workflow, stating "the current assistant is `assistant_name`".

from typing import Callable

from langchain_core.messages import ToolMessage


def create_entry_node(assistant_name: str, new_dialog_state: str) -> Callable:
    def entry_node(state: State) -> dict:
        tool_call_id = state["messages"][-1].tool_calls[0]["id"]
        return {
            "messages": [
                ToolMessage(
                    content=f"The assistant is now the {assistant_name}. Reflect on the above conversation between the host assistant and the user."
                    f" The user's intent is unsatisfied. Use the provided tools to assist the user. Remember, you are {assistant_name},"
                    " and the booking, update, other other action is not complete until after you have successfully invoked the appropriate tool."
                    " If the user changes their mind or needs help for other tasks, call the CompleteOrEscalate function to let the primary host assistant take control."
                    " Do not mention who you are - just act as the proxy for the assistant.",
                    tool_call_id=tool_call_id,
                )
            ],
            "dialog_state": new_dialog_state,
        }

    return entry_node

#### Define Graph

Now it's time to start building our graph. As before, we'll start with a node to pre-populate the state with the user's current information.

from typing import Literal

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)


def user_info(state: State):
    return {"user_info": fetch_user_flight_information.invoke({})}


builder.add_node("fetch_user_info", user_info)
builder.add_edge(START, "fetch_user_info")

Now we'll start adding our specialized workflows. Each mini-workflow looks very similar to our full graph in [Part 3](#part-3-conditional-interrupt), employing 5 nodes:

1. `enter_*`: use the `create_entry_node` utility you defined above to add a ToolMessage signaling that the new specialized assistant is at the helm
2. Assistant: the prompt + llm combo that takes in the current state and either uses a tool, asks a question of the user, or ends the workflow (return to the primary assistant)
3. `*_safe_tools`: "read-only" tools the assistant can use without user confirmation.
4. `*_sensitive_tools`: tools with "write" access that require user confirmation (and will be assigned an `interrupt_before` when we compile the graph)
5. `leave_skill`: _pop_ the `dialog_state` to signal that the *primary assistant* is back in control

Because of their similarities, we _could_ define a factory function to generate these. Since this is a tutorial, we'll define them each explicitly.

First, make the **flight booking assistant** dedicated to managing the user journey for updating and canceling flights.

# Flight booking assistant
builder.add_node(
    "enter_update_flight",
    create_entry_node("Flight Updates & Booking Assistant", "update_flight"),
)
builder.add_node("update_flight", Assistant(update_flight_runnable))
builder.add_edge("enter_update_flight", "update_flight")
builder.add_node(
    "update_flight_sensitive_tools",
    create_tool_node_with_fallback(update_flight_sensitive_tools),
)
builder.add_node(
    "update_flight_safe_tools",
    create_tool_node_with_fallback(update_flight_safe_tools),
)


def route_update_flight(
    state: State,
):
    route = tools_condition(state)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    did_cancel = any(tc["name"] == CompleteOrEscalate.__name__ for tc in tool_calls)
    if did_cancel:
        return "leave_skill"
    safe_toolnames = [t.name for t in update_flight_safe_tools]
    if all(tc["name"] in safe_toolnames for tc in tool_calls):
        return "update_flight_safe_tools"
    return "update_flight_sensitive_tools"


builder.add_edge("update_flight_sensitive_tools", "update_flight")
builder.add_edge("update_flight_safe_tools", "update_flight")
builder.add_conditional_edges(
    "update_flight",
    route_update_flight,
    ["update_flight_sensitive_tools", "update_flight_safe_tools", "leave_skill", END],
)


# This node will be shared for exiting all specialized assistants
def pop_dialog_state(state: State) -> dict:
    """Pop the dialog stack and return to the main assistant.

    This lets the full graph explicitly track the dialog flow and delegate control
    to specific sub-graphs.
    """
    messages = []
    if state["messages"][-1].tool_calls:
        # Note: Doesn't currently handle the edge case where the llm performs parallel tool calls
        messages.append(
            ToolMessage(
                content="Resuming dialog with the host assistant. Please reflect on the past conversation and assist the user as needed.",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        )
    return {
        "dialog_state": "pop",
        "messages": messages,
    }


builder.add_node("leave_skill", pop_dialog_state)
builder.add_edge("leave_skill", "primary_assistant")

Next, create the **car rental assistant** graph to own all car rental needs.

# Car rental assistant

builder.add_node(
    "enter_book_car_rental",
    create_entry_node("Car Rental Assistant", "book_car_rental"),
)
builder.add_node("book_car_rental", Assistant(book_car_rental_runnable))
builder.add_edge("enter_book_car_rental", "book_car_rental")
builder.add_node(
    "book_car_rental_safe_tools",
    create_tool_node_with_fallback(book_car_rental_safe_tools),
)
builder.add_node(
    "book_car_rental_sensitive_tools",
    create_tool_node_with_fallback(book_car_rental_sensitive_tools),
)


def route_book_car_rental(
    state: State,
):
    route = tools_condition(state)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    did_cancel = any(tc["name"] == CompleteOrEscalate.__name__ for tc in tool_calls)
    if did_cancel:
        return "leave_skill"
    safe_toolnames = [t.name for t in book_car_rental_safe_tools]
    if all(tc["name"] in safe_toolnames for tc in tool_calls):
        return "book_car_rental_safe_tools"
    return "book_car_rental_sensitive_tools"


builder.add_edge("book_car_rental_sensitive_tools", "book_car_rental")
builder.add_edge("book_car_rental_safe_tools", "book_car_rental")
builder.add_conditional_edges(
    "book_car_rental",
    route_book_car_rental,
    [
        "book_car_rental_safe_tools",
        "book_car_rental_sensitive_tools",
        "leave_skill",
        END,
    ],
)

Then define the **hotel booking** workflow.

# Hotel booking assistant
builder.add_node(
    "enter_book_hotel", create_entry_node("Hotel Booking Assistant", "book_hotel")
)
builder.add_node("book_hotel", Assistant(book_hotel_runnable))
builder.add_edge("enter_book_hotel", "book_hotel")
builder.add_node(
    "book_hotel_safe_tools",
    create_tool_node_with_fallback(book_hotel_safe_tools),
)
builder.add_node(
    "book_hotel_sensitive_tools",
    create_tool_node_with_fallback(book_hotel_sensitive_tools),
)


def route_book_hotel(
    state: State,
):
    route = tools_condition(state)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    did_cancel = any(tc["name"] == CompleteOrEscalate.__name__ for tc in tool_calls)
    if did_cancel:
        return "leave_skill"
    tool_names = [t.name for t in book_hotel_safe_tools]
    if all(tc["name"] in tool_names for tc in tool_calls):
        return "book_hotel_safe_tools"
    return "book_hotel_sensitive_tools"


builder.add_edge("book_hotel_sensitive_tools", "book_hotel")
builder.add_edge("book_hotel_safe_tools", "book_hotel")
builder.add_conditional_edges(
    "book_hotel",
    route_book_hotel,
    ["leave_skill", "book_hotel_safe_tools", "book_hotel_sensitive_tools", END],
)

After that, define the **excursion assistant**.

# Excursion assistant
builder.add_node(
    "enter_book_excursion",
    create_entry_node("Trip Recommendation Assistant", "book_excursion"),
)
builder.add_node("book_excursion", Assistant(book_excursion_runnable))
builder.add_edge("enter_book_excursion", "book_excursion")
builder.add_node(
    "book_excursion_safe_tools",
    create_tool_node_with_fallback(book_excursion_safe_tools),
)
builder.add_node(
    "book_excursion_sensitive_tools",
    create_tool_node_with_fallback(book_excursion_sensitive_tools),
)


def route_book_excursion(
    state: State,
):
    route = tools_condition(state)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    did_cancel = any(tc["name"] == CompleteOrEscalate.__name__ for tc in tool_calls)
    if did_cancel:
        return "leave_skill"
    tool_names = [t.name for t in book_excursion_safe_tools]
    if all(tc["name"] in tool_names for tc in tool_calls):
        return "book_excursion_safe_tools"
    return "book_excursion_sensitive_tools"


builder.add_edge("book_excursion_sensitive_tools", "book_excursion")
builder.add_edge("book_excursion_safe_tools", "book_excursion")
builder.add_conditional_edges(
    "book_excursion",
    route_book_excursion,
    ["book_excursion_safe_tools", "book_excursion_sensitive_tools", "leave_skill", END],
)

Finally, create the **primary assistant**.

# Primary assistant
builder.add_node("primary_assistant", Assistant(assistant_runnable))
builder.add_node(
    "primary_assistant_tools", create_tool_node_with_fallback(primary_assistant_tools)
)


def route_primary_assistant(
    state: State,
):
    route = tools_condition(state)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    if tool_calls:
        if tool_calls[0]["name"] == ToFlightBookingAssistant.__name__:
            return "enter_update_flight"
        elif tool_calls[0]["name"] == ToBookCarRental.__name__:
            return "enter_book_car_rental"
        elif tool_calls[0]["name"] == ToHotelBookingAssistant.__name__:
            return "enter_book_hotel"
        elif tool_calls[0]["name"] == ToBookExcursion.__name__:
            return "enter_book_excursion"
        return "primary_assistant_tools"
    raise ValueError("Invalid route")


# The assistant can route to one of the delegated assistants,
# directly use a tool, or directly respond to the user
builder.add_conditional_edges(
    "primary_assistant",
    route_primary_assistant,
    [
        "enter_update_flight",
        "enter_book_car_rental",
        "enter_book_hotel",
        "enter_book_excursion",
        "primary_assistant_tools",
        END,
    ],
)
builder.add_edge("primary_assistant_tools", "primary_assistant")


# Each delegated workflow can directly respond to the user
# When the user responds, we want to return to the currently active workflow
def route_to_workflow(
    state: State,
) -> Literal[
    "primary_assistant",
    "update_flight",
    "book_car_rental",
    "book_hotel",
    "book_excursion",
]:
    """If we are in a delegated state, route directly to the appropriate assistant."""
    dialog_state = state.get("dialog_state")
    if not dialog_state:
        return "primary_assistant"
    return dialog_state[-1]


builder.add_conditional_edges("fetch_user_info", route_to_workflow)

# Compile graph
memory = MemorySaver()
part_4_graph = builder.compile(
    checkpointer=memory,
    # Let the user approve or deny the use of sensitive tools
    interrupt_before=[
        "update_flight_sensitive_tools",
        "book_car_rental_sensitive_tools",
        "book_hotel_sensitive_tools",
        "book_excursion_sensitive_tools",
    ],
)

from IPython.display import Image, display

try:
    display(Image(part_4_graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

#### Conversation

That was a lot! Let's run it over the following list of dialog turns. This time, we'll have many fewer confirmations.

import shutil
import uuid

# Update with the backup file so we can restart from the original place in each section
db = update_dates(db)
thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "passenger_id": "3442 587242",
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

_printed = set()
# We can reuse the tutorial questions from part 1 to see how it does.
for question in tutorial_questions:
    events = part_4_graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)
    snapshot = part_4_graph.get_state(config)
    while snapshot.next:
        # We have an interrupt! The agent is trying to use a tool, and the user can approve or deny it
        # Note: This code is all outside of your graph. Typically, you would stream the output to a UI.
        # Then, you would have the frontend trigger a new run via an API call when the user has provided input.
        try:
            user_input = input(
                "Do you approve of the above actions? Type 'y' to continue;"
                " otherwise, explain your requested changed.\n\n"
            )
        except:
            user_input = "y"
        if user_input.strip() == "y":
            # Just continue
            result = part_4_graph.invoke(
                None,
                config,
            )
        else:
            # Satisfy the tool invocation by
            # providing instructions on the requested changes / change of mind
            result = part_4_graph.invoke(
                {
                    "messages": [
                        ToolMessage(
                            tool_call_id=event["messages"][-1].tool_calls[0]["id"],
                            content=f"API call denied by user. Reasoning: '{user_input}'. Continue assisting, accounting for the user's input.",
                        )
                    ]
                },
                config,
            )
        snapshot = part_4_graph.get_state(config)

#### Conclusion:

You've now developed a customer support bot that handles diverse tasks using focused workflows.
More importantly, you've learned to use some of LangGraph's core features to design and refactor an application based on your product needs.

The above examples are by no means optimized for your unique needs - LLMs make mistakes, and each flow can be made more reliable through better prompts and experimentation. Once you've created your initial support bot, the next step would be to start [adding evaluations](https://docs.smith.langchain.com/evaluation) so you can confidently improve your system. Check out those docs and our other tutorials to learn more! -->